# Sampling Data

The purpose of this notebook is to sample 10% of the training observations and then perform feature selection. This is necessary because the large size of the full feature matrices does not allow for efficient feature selection or for a significant number of random search iterations.

## Roadmap

1. Sample 10% of the training observations randomly
2. Convert numeric columns to `np.float32`
3. Convert boolean columns to `np.uint8`
4. One-hot encode categorical features as necessary
5. Remove one of every pair of columns with all duplicated values (1.0 correlation)
6. Remove columns with more than 90% missing values
7. Remove columns with a single unique value
8. Remove one of every pair of columns with abs(correlation) > 0.95

The reduced data set will then be saved as `_sample.csv` and can be used for 100 iterations of random search with the Gradient Boosting Machine.

In [1]:
import pandas as pd
import numpy as np

/home/ubuntu/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/ubuntu/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


# Featuretools Features

In [2]:
feature_matrix = pd.read_csv('../input/feature_matrix.csv', low_memory=False)

# Sampling 10% of the original data
train = feature_matrix[feature_matrix['TARGET'].notnull()].sample(frac = 0.1, random_state = 50)

import gc
gc.enable()
del feature_matrix
gc.collect()

0

### Correct column types

In [3]:
for col in ['SUM(bureau.PREVIOUS_OTHER_LOAN_RATE)', 'SUM(bureau.PREVIOUS_OTHER_LOAN_RATE WHERE CREDIT_ACTIVE = Closed)',
            'SUM(bureau.PREVIOUS_OTHER_LOAN_RATE WHERE CREDIT_ACTIVE = Active)', 'SUM(bureau_balance.bureau.PREVIOUS_OTHER_LOAN_RATE)']:
    try:
        train[col] = train[col].astype(np.float32)
    except:
        print(f'{col} not in data')
    
for col in train:
    if train[col].dtype == 'bool':
        train[col] = train[col].astype(np.uint8)

SUM(bureau.PREVIOUS_OTHER_LOAN_RATE) not in data
SUM(bureau.PREVIOUS_OTHER_LOAN_RATE WHERE CREDIT_ACTIVE = Closed) not in data
SUM(bureau.PREVIOUS_OTHER_LOAN_RATE WHERE CREDIT_ACTIVE = Active) not in data
SUM(bureau_balance.bureau.PREVIOUS_OTHER_LOAN_RATE) not in data


In [4]:
train = pd.get_dummies(train)
n_features_start = train.shape[1] - 2
train.shape

(30751, 2091)

### Columns with duplicated values

In [5]:
x, idx, inv, counts = np.unique(train, axis = 1, return_index = True, return_inverse=True, return_counts=True)
train = train.iloc[:, idx]
n_non_unique_columns = n_features_start - train.shape[1] - 2
train.shape

(30751, 1816)

### Missing Values

The threshold is currently set at 90% but could be lowered. 

In [6]:
missing_threshold = 90

# Find missing and percentage
missing = pd.DataFrame(train.isnull().sum())
missing['percent'] = 100 * (missing[0] / train.shape[0])
missing.sort_values('percent', ascending = False, inplace = True)

# Missing above threshold
missing_cols = list(missing[missing['percent'] > missing_threshold].index)
n_missing_cols = len(missing_cols)

train = train[[x for x in train if x not in missing_cols]]
train.shape

(30751, 1798)

### Zero variance columns

These are any columns with only a single unique value. (`np.nan` does not count as a unique value)

In [7]:
unique_counts = pd.DataFrame(train.nunique()).sort_values(0, ascending = True)
zero_variance_cols = list(unique_counts[unique_counts[0] == 1].index)
n_zero_variance_cols = len(zero_variance_cols)

train = train[[x for x in train if x not in zero_variance_cols]]
train.shape

(30751, 1771)

#### Remove columns containing derivations of target

This is a special consideration we have to take with the Featuretools data because one of the columns is derived from the `TARGET`. (PERCENTILE transformation works on numeric columns of the data and I accidentally left in the `TARGET` when running the feature synthesis).

In [8]:
for col in train:
    if 'TARGET' in col:
        print(col)
        
train.drop(columns = 'PERCENTILE(TARGET)', inplace = True)

TARGET
PERCENTILE(TARGET)


### Find Collinear Variables with Correlation Threshold

The correlation threshold will be set at 0.95 and one out of every pair of columns that are above this threshold will be removed. The column removed is the one that occurs last in the data.

In [9]:
correlation_threshold = 0.95

corr_matrix = train.corr()

# Extract the upper triangle of the correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k = 1).astype(np.bool))

# Select the features with correlations above the threshold
# Need to use the absolute value
to_drop = [column for column in upper.columns if any(upper[column].abs() > correlation_threshold)]

In [10]:
train = train[[x for x in train if x not in to_drop]]
n_collinear = len(to_drop)
train.shape

(30751, 1042)

In [11]:
n_non_unique_columns

271

In [12]:
n_missing_cols

18

In [13]:
n_zero_variance_cols

27

In [14]:
n_collinear

728

In [15]:
total_removed = n_non_unique_columns + n_missing_cols + n_zero_variance_cols + n_collinear + 1
print('Total columns removed: ', total_removed)

Total columns removed:  1045


In [16]:
train.head()

,DAYS_BIRTH,DAYS_REGISTRATION,SUM(installments.DAYS_ENTRY_PAYMENT),MIN(previous.SUM(installments.DAYS_ENTRY_PAYMENT)),MEAN(previous.SUM(installments.DAYS_ENTRY_PAYMENT)),MAX(previous.SUM(installments.DAYS_ENTRY_PAYMENT)),DAYS_EMPLOYED,DAYS_ID_PUBLISH,SUM(previous.DAYS_DECISION),MIN(previous.DAYS_DECISION),...,PERCENTILE(MEAN(bureau.AMT_ANNUITY)),MIN(bureau.PERCENTILE(AMT_ANNUITY)),MEAN(bureau.PERCENTILE(AMT_ANNUITY)),MIN(bureau.NUM_UNIQUE(bureau_balance.STATUS)),MEAN(bureau.NUM_UNIQUE(bureau_balance.STATUS)),MAX(bureau.NUM_UNIQUE(bureau_balance.STATUS)),NUM_UNIQUE(bureau_balance.STATUS),MIN(bureau.AMT_ANNUITY),MEAN(bureau.AMT_ANNUITY),MAX(bureau.AMT_ANNUITY)
83147,-11819.0,-4187.0,-726.0,-726.0,-726.000000,-726.0,-673.0,-601.0,-217.0,-217.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
354935,-18794.0,-6010.0,-157073.0,-122152.0,-39268.250000,-1026.0,-8164.0,-2201.0,-8634.0,-2648.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
64916,-20741.0,-1882.0,-20122.0,-13200.0,-10061.000000,-6922.0,NaN,-4296.0,-2900.0,-2041.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
87122,-9881.0,-932.0,NaN,NaN,NaN,NaN,-185.0,-1076.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
280351,-14762.0,-6189.0,-57878.0,-28635.0,-19292.666667,-10697.0,-2370.0,-4187.0,-7679.0,-2745.0,...,0.661964,0.706131,0.706131,3.0,3.0,3.0,3.0,10984.5,10984.5,10984.5


## Save sample of Data

The resulting data has 10% of the training observations and 1042 columns (1040 of which are features). This data is now ready for random search.

In [18]:
train.to_csv('../input/feature_matrix_sample.csv', index = False)

# Function for Feature Selection

We can refactor the four steps completed above into a single function that applies them in the same sequence to any dataframe. This function will be used on the manual and semi-automated feaures with the same inputs as with the Featuretools features.

In [ ]:
def feature_selection(feature_matrix, missing_threshold=90, correlation_threshold=0.95):
    """Feature selection for a dataframe."""
    
    feature_matrix = pd.get_dummies(feature_matrix)
    n_features_start = feature_matrix.shape[1]
    print('Original shape: ', feature_matrix.shape)

    _, idx = np.unique(feature_matrix, axis = 1, return_index = True)
    feature_matrix = feature_matrix.iloc[:, idx]
    n_non_unique_columns = n_features_start - feature_matrix.shape[1]
    print('{}  non-unique valued columns.'.format(n_non_unique_columns))

    # Find missing and percentage
    missing = pd.DataFrame(feature_matrix.isnull().sum())
    missing['percent'] = 100 * (missing[0] / feature_matrix.shape[0])
    missing.sort_values('percent', ascending = False, inplace = True)

    # Missing above threshold
    missing_cols = list(missing[missing['percent'] > missing_threshold].index)
    n_missing_cols = len(missing_cols)

    # Remove missing columns
    feature_matrix = feature_matrix[[x for x in feature_matrix if x not in missing_cols]]
    print('{} missing columns with threshold: {}.'.format(n_missing_cols,
                                                                        missing_threshold))
    
    # Zero variance
    unique_counts = pd.DataFrame(feature_matrix.nunique()).sort_values(0, ascending = True)
    zero_variance_cols = list(unique_counts[unique_counts[0] == 1].index)
    n_zero_variance_cols = len(zero_variance_cols)

    # Remove zero variance columns
    feature_matrix = feature_matrix[[x for x in feature_matrix if x not in zero_variance_cols]]
    print('{} zero variance columns.'.format(n_zero_variance_cols))
    
    # Correlations
    corr_matrix = feature_matrix.corr()

    # Extract the upper triangle of the correlation matrix
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k = 1).astype(np.bool))

    # Select the features with correlations above the threshold
    # Need to use the absolute value
    to_drop = [column for column in upper.columns if any(upper[column].abs() > correlation_threshold)]

    n_collinear = len(to_drop)
    
    feature_matrix = feature_matrix[[x for x in feature_matrix if x not in to_drop]]
    print('{} collinear columns removed with threshold: {}.'.format(n_collinear,
                                                                          correlation_threshold))
    
    total_removed = n_non_unique_columns + n_missing_cols + n_zero_variance_cols + n_collinear
    
    print('Total columns removed: ', total_removed)
    print('Shape after feature selection: {}.'.format(feature_matrix.shape))
    return feature_matrix

# Manual Features

The process is the same except now we have a function to carry out the feature selection. First we subset to 10% of the training data, and then we apply feature selection. 

In [ ]:
manual_features = pd.read_csv('../input/features_manual.csv')
manual_features = manual_features[manual_features['TARGET'].notnull()].sample(frac = 0.1, random_state = 50)

manual_features = feature_selection(manual_features, 90, 0.95)

In [ ]:
manual_features.head()

Then we can save the features for random search. 

In [ ]:
manual_features.to_csv('../input/features_manual_sample.csv', index = False)

# Semi-Automated Features

The final feature matrix is that created by what we called __semi-automated__ feature engineering. The same exact process applies.

In [ ]:
semi_features = pd.read_csv('../input/features_semi.csv')
semi_features = semi_features[semi_features['TARGET'].notnull()].sample(frac = 0.1, random_state = 50)

semi_features = feature_selection(semi_features, 90, 0.95)

In [ ]:
semi_features.head()

In [ ]:
semi_features.to_csv('../input/features_semi_sample.csv', index = False)

# Default Features

These are the features available in the main dataframe.

In [ ]:
fm = pd.read_csv('../input/application_train.csv')
fm = fm.sample(frac = 0.1, random_state = 50)

fm = pd.get_dummies(fm)
fm.shape

In [ ]:
fm = feature_selection(fm, 90, 0.95)
fm.head()

In [ ]:
fm.to_csv('../input/features_default_sample.csv', index = False)

# Conclusions

In order to allow us to perform feature selection, we first had to limit the number of rows of data to 10% of the training observations. Then we were able to apply feature selection to reduce the data dimensionality. The final sampled and selected data was saved as `_sample.csv` and can now be used for random search with the Gradient Boosting Machine. There are a number of "arbitrary" thresholds in this approach, but by applying the same operations to all 3 datasets, it is hoped that these choices will not affect the integrity of the analysis. At the end of the day, some results are better than no results and these operations will allow us to perform 100 iterations of random search and proceed with the project. 

The next step is to run random search on these results. This is implemented in the `random_search.py` script in the scripts directory.

The next notebook is Subsetting Data where we use these results in order to create versions of the feature matrices with all the observations but only the columns identified in this notebook. 